In [1]:
import os, re, csv
import subprocess
import numpy as np
import pandas as pd
import librosa
import sounddevice as sd
import ccpreprocess, ccdownload, ccwebcrawl, ccchord
import torch
from torch.utils.data import Dataset

dir_path   = os.path.abspath('')
excel_path = os.path.join(dir_path, 'ccClub music data (popular music).xlsx')
excel_data = pd.read_excel(excel_path)

excel_data.head(3)

,Title,Link,Start_Second,Tempo,Capo,Tune,3 or 4,Chords
0,刻在我心底的名字,https://www.youtube.com/watch?v=2sX45Oyd1n0,0.0,68.0,3.0,G,4.0,Gadd9|Em7|Cmaj7|Cm|Gadd9|Em7|Am7|Cm|Gadd9|F#m7...
1,飛鳥和蟬,https://www.youtube.com/watch?v=V0XUd8f2pz8,3.0,75.0,3.0,G,4.0,G|D/F#|Em7|Cmaj7|G|D/F#|Em7|Cmaj7|G|Bm7|Em7|Bm...
2,晴天,https://www.youtube.com/watch?v=v10bFxUDZsA,0.0,70.0,0.0,G,4.0,"Em7,Cadd9|G,D/F#|Em7,Cadd9|G,D/F#|Em7,Cadd9|G,..."


In [2]:
count = 0
for chord in excel_data['Chords']:
    chord = chord.strip('|')
    chord = chord.replace(' ', '')
    chord = chord.split('|')
    for f in range(len(chord)):
        chord[f] = chord[f].split(',')
        for i in range(len(chord[f])):
            chord[f][i] = ccchord.chord_simplify(chord[f][i])
            try:
                x = ccchord.chord_numeralize(excel_data['Capo'][count], chord[f][i])
            except:
                print(chord)
                print(count, excel_data['Title'][count])
            print(x)
    count += 1

[0.         0.         0.57735027 0.         0.         0.57735027
 0.         0.         0.         0.         0.57735027 0.        ]
[0.  0.  0.5 0.  0.  0.5 0.  0.5 0.  0.  0.5 0. ]
[0.  0.  0.5 0.5 0.  0.  0.  0.5 0.  0.  0.5 0. ]
[0.         0.         0.         0.57735027 0.         0.
 0.57735027 0.         0.         0.         0.57735027 0.        ]
[0.         0.         0.57735027 0.         0.         0.57735027
 0.         0.         0.         0.         0.57735027 0.        ]
[0.  0.  0.5 0.  0.  0.5 0.  0.5 0.  0.  0.5 0. ]
[0.5 0.  0.  0.5 0.  0.  0.  0.5 0.  0.  0.5 0. ]
[0.         0.         0.         0.57735027 0.         0.
 0.57735027 0.         0.         0.         0.57735027 0.        ]
[0.         0.         0.57735027 0.         0.         0.57735027
 0.         0.         0.         0.         0.57735027 0.        ]
[0.5 0.  0.  0.  0.5 0.  0.  0.5 0.  0.5 0.  0. ]
[0.5 0.  0.5 0.  0.  0.  0.5 0.  0.  0.5 0.  0. ]
[0.  0.  0.5 0.  0.  0.5 0.  0.5 0.  0.  

In [3]:
for row in range(excel_data.shape[0]):
    wav_path = os.path.join(dir_path, f"music/{excel_data['Title'][row]}.wav")
    if not os.path.isfile(wav_path):
        ccdownload.yt_wav(link=excel_data['Link'][row], file_name=excel_data['Title'][row])

In [ ]:
class MusicDataset(Dataset):
    def __init__(self, wav, start_second, tempo, capo, tune)